<a href="https://colab.research.google.com/github/ckpraneet3t/DenseRetrieval_Llama/blob/main/Nvidia_Incois_(chunks).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyMuPDF
!pip install sentence-transformers
!pip install faiss-cpu
!pip install transformers optimum torch accelerate auto-gptq


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinu

In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

def split_into_chunks(text, chunk_size=500):
    words = text.split()
    chunks = [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

pdf_path = '/content/INCOIS_Chatbot_Dataset.pdf'
bike_data_text = extract_text_from_pdf(pdf_path)
chunks = split_into_chunks(bike_data_text)
print(f"Number of chunks: {len(chunks)}")


Number of chunks: 906


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load a pre-trained sentence transformer model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Embed the text chunks
embeddings = embedding_model.encode(chunks)

# Create a FAISS index and add the embeddings
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings).astype(np.float32))

# Save the index and chunks for later use
faiss.write_index(index, 'incois_data_chunks.index')
np.save('incois_chunks.npy', np.array(chunks))


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline, AutoTokenizer

model_name = "TheBloke/Llama-2-13B-chat-GPTQ"
llama_pipe = pipeline("text-generation", model=model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

def generate(prompt, max_length=4096, pipe=llama_pipe, **kwargs):
    def_kwargs = dict(return_full_text=False, return_dict=False)
    response = pipe(prompt.strip(), max_length=max_length, **kwargs, **def_kwargs)
    return response[0]['generated_text'].strip()

def construct_prompt_with_context(main_prompt, system_context="", conversation_examples=[]):
    if not system_context and not conversation_examples:
        return main_prompt

    full_prompt = f"<s>[INST] <<SYS>>{system_context}<</SYS>>\n" if system_context else "<s>[INST]\n"

    for user_msg, agent_response in conversation_examples:
        full_prompt += f"{user_msg} [/INST] {agent_response} </s><s>[INST]"

    full_prompt += f"{main_prompt} [/INST]"

    return full_prompt

class LlamaChatbot:
    def __init__(self, system_context):
        self.system_context = system_context
        self.conversation_history = []
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.index = faiss.read_index('incois_data_chunks.index')
        self.chunks = np.load('incois_chunks.npy', allow_pickle=True).tolist()

    def chat(self, user_msg):
        prompt = construct_prompt_with_context(user_msg, self.system_context, self.conversation_history)
        similar_chunks = self.search_similar_chunks(user_msg)
        context = '\n'.join(similar_chunks)
        full_prompt = f"{self.system_context}\n{context}\n{prompt}"
        agent_response = generate(full_prompt)
        self.conversation_history.append((user_msg, agent_response))
        return agent_response

    def search_similar_chunks(self, query):
        query_embedding = self.embedding_model.encode([query])
        D, I = self.index.search(np.array(query_embedding).astype(np.float32), k=3)
        return [self.chunks[i] for i in I[0]]

    def reset(self):
        self.conversation_history = []

system_context = """
You are an AI assistant knowledgeable about oceanographic and marine information services provided by INCOIS.
When asked about specific services, data, or features, you provide accurate and helpful information.
Your goal is to assist and inform users to the best of your ability in a formal and concise manner.
"""


config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Llama-2-13B-chat-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.ml

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
chatbot = LlamaChatbot(system_context)

print(chatbot.chat("What are the sources of getting a vertical profiles?"))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1283: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Greetings! I'm happy to assist you with information on oceanographic and marine data services provided by INCOIS.

Regarding vertical profiles, there are several sources that can be used to obtain this information:

1. Argo Floats: INCOIS operates a network of Argo floats that measure the physical and biogeochemical properties of the ocean, including temperature, salinity, and chlorophyll. These floats can provide vertical profiles of the ocean up to 1000 m depth.
2. XBTs (Expendable Bathythermographs): XBTs are instruments that measure the temperature of the ocean as they are lowered into the water. They can provide vertical profiles of temperature up to 1000 m depth.
3. CTDs (Conductivity, Temperature, and Depth) instruments: CTDs are instruments that measure conductivity, temperature, and depth. They can provide vertical profiles of temperature, salinity, and other properties up to 1000 m depth.
4. ADCPs (Acoustic Doppler Current Profilers): ADCPs are instruments that measure the cu

In [ ]:
from IPython import get_ipython

get_ipython().kernel.do_shutdown(restart=True)